In [1]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.linalg
import sklearn.linear_model
import sklearn.preprocessing

# Part (c): Read in the data, standardize it
spam = pd.read_table('https://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/spam.data', sep=' ', header=None)
test_indicator = pd.read_table('https://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/spam.traintest', sep=' ',
                               header=None)

x = np.asarray(spam)[:, 0:-1]
y = np.asarray(spam)[:, -1]*2 - 1  # Convert to +/- 1
test_indicator = np.array(test_indicator).T[0]

# Divide the data into train, test sets
x_train = x[test_indicator == 0, :]
x_test = x[test_indicator == 1, :]
y_train = y[test_indicator == 0]
y_test = y[test_indicator == 1]

# Standardize the data.
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

# Keep track of the number of samples and dimension of each sample
n_train = len(y_train)
n_test = len(y_test)
d = np.size(x, 1)

# Write a function computegrad that computes and returns rF( ) for any  .
def computegrad(beta, lambduh, x=x_train, y=y_train):
    yx = y[:, np.newaxis]*x
    denom = 1+np.exp(-yx.dot(beta))
    grad = 1/len(y)*np.sum(-yx*np.exp(-yx.dot(beta[:, np.newaxis]))/denom[:, np.newaxis], axis=0) + 2*lambduh*beta
    return grad

# Write a function backtracking that implements the backtracking rule.
def objective(beta, lambduh, x=x_train, y=y_train):
    return 1/len(y) * np.sum(np.log(1 + np.exp(-y*x.dot(beta)))) + lambduh * np.linalg.norm(beta)**2

# Write a function graddescent that implements the gradient descent algorithm with the backtracking rule to tune the step-size. The function graddescent calls computegrad and backtracking as subroutines. The function takes as input the initial point, the initial step-size value, and the maximum number of iterations. The stopping criterion is the maximum number of iterations.
def bt_line_search(beta, lambduh, eta=1, alpha=0.5, betaparam=0.8,
                   maxiter=100, x=x_train, y=y_train):
    grad_beta = computegrad(beta, lambduh, x=x, y=y)
    norm_grad_beta = np.linalg.norm(grad_beta)
    found_eta = 0
    iter = 0
    while found_eta == 0 and iter < maxiter:
        if objective(beta - eta * grad_beta, lambduh, x=x, y=y) < objective(beta, lambduh, x=x, y=y) \
                - alpha * eta * norm_grad_beta ** 2:
            found_eta = 1
        elif iter == maxiter:
            raise ('Max number of iterations of backtracking'
                   ' line search reached')
        else:
            eta *= betaparam
            iter += 1
    return eta


# Part (f): Gradient descent algorithm
def graddescent(beta_init, lambduh, eta_init, maxiter, x=x_train, y=y_train):
    beta = beta_init
    grad_beta = computegrad(beta, lambduh, x=x, y=y)
    beta_vals = beta
    iter = 0
    while iter < maxiter:
        eta = bt_line_search(beta, lambduh, eta=eta_init, x=x, y=y)
        beta = beta - eta*grad_beta
        # Store all of the places we step to
        beta_vals = np.vstack((beta_vals, beta))
        grad_beta = computegrad(beta, lambduh, x=x, y=y)
        iter += 1
        if iter % 100 == 0:
            print('Gradient descent iteration', iter)
    return beta_vals

# Write a function fastgradalgo that implements the fast gradient algorithm described in Algorithm 1. The function fastgradalgo calls computegrad and backtracking as sub- routines. The function takes as input the initial point, the initial step-size value, and the maximum number of iterations. The stopping criterion is the maximum number of iterations.
def fastgradalgo(beta_init, theta_init, lambduh, eta_init, maxiter, x=x_train, y=y_train):
    beta = beta_init
    theta = theta_init
    grad_theta = computegrad(theta, lambduh, x=x, y=y)
    theta_vals = theta
    iter = 0
    while iter < maxiter:
        eta = bt_line_search(theta, lambduh, eta=eta_init, x=x, y=y)
        beta_new = theta - eta*grad_theta
        theta = beta_new + iter/(iter+3)*(beta_new-beta)
        # Store all of the places we step to
        theta_vals = np.vstack((theta_vals, theta))
        grad_theta = computegrad(theta, lambduh, x=x, y=y)
        beta = beta_new
        iter += 1
        if iter % 100 == 0:
            print('Fast gradient iteration', iter)
    return theta_vals

# Use the estimate described in the course to initialize the step-size. Set the maximum number of iterations to 1000. Run graddescent and fastgradalgo on the training set of the Spam dataset for   = 0.1. Plot the curve of the objective values F( t) for both algorithms versus the iteration counter t (use different colors). What do you observe?
def objective_plot(betas_gd, betas_fg, lambduh, x=x_train, y=y_train, save_file=''):
    num_points = np.size(betas_gd, 0)
    objs_gd = np.zeros(num_points)
    objs_fg = np.zeros(num_points)
    for i in range(0, num_points):
        objs_gd[i] = objective(betas_gd[i, :], lambduh, x=x, y=y)
        objs_fg[i] = objective(betas_fg[i, :], lambduh, x=x, y=y)
    fig, ax = plt.subplots()
    ax.plot(range(1, num_points + 1), objs_gd, label='gradient descent')
    ax.plot(range(1, num_points + 1), objs_fg, c='red', label='fast gradient')
    plt.xlabel('Iteration')
    plt.ylabel('Objective value')
    plt.title('Objective value vs. iteration when lambda='+str(lambduh))
    ax.legend(loc='upper right')
    if not save_file:
        plt.show()
    else:
        plt.savefig(save_file)


lambduh = 0.1
beta_init = np.zeros(d)
theta_init = np.zeros(d)
eta_init = 1/(scipy.linalg.eigh(1/len(y_train)*x_train.T.dot(x_train), eigvals=(d-1, d-1), eigvals_only=True)[0]+lambduh)
maxiter = 1000
betas_grad = graddescent(beta_init, lambduh, eta_init, maxiter)
betas_fastgrad = fastgradalgo(beta_init, theta_init, lambduh, eta_init, maxiter)
objective_plot(betas_grad, betas_fastgrad, lambduh, save_file='hw3_q1_part_h_output.png')

#Denote by  T the final iterate of your fast gradient algorithm. Compare  T to the  ? found by scikit-learn. Compare the objective value for  T to the one for  ?. What do you observe?
lr = sklearn.linear_model.LogisticRegression(penalty='l2', C=1 / (2 * lambduh * n_train), fit_intercept=False,
                                             tol=10e-8, max_iter=1000)
lr.fit(x_train, y_train)
print(lr.coef_)
print(betas_fastgrad[-1, :])

print(objective(betas_fastgrad[-1, :], lambduh))
print(objective(lr.coef_.flatten(), lambduh))


# Run cross-validation on the training set of the Spam dataset using scikit-learn to find the optimal value of  . Run graddescent and fastgradalgo to optimize the objective with that value of  . Plot the curve of the objective values F( t) for both algorithms versus the iteration counter t. Plot the misclassification error on the training set for both algorithms versus the iteration counter t. Plot the misclassification error on the test set for both algorithms versus the iteration counter t. What do you observe?
def compute_misclassification_error(beta_opt, x, y):
    y_pred = 1/(1+np.exp(-x.dot(beta_opt))) > 0.5
    y_pred = y_pred*2 - 1  # Convert to +/- 1
    return np.mean(y_pred != y)


def plot_misclassification_error(betas_grad, betas_fastgrad, x, y, save_file='', title=''):
    niter = np.size(betas_grad, 0)
    error_grad = np.zeros(niter)
    error_fastgrad = np.zeros(niter)
    for i in range(niter):
        error_grad[i] = compute_misclassification_error(betas_grad[i, :], x, y)
        error_fastgrad[i] = compute_misclassification_error(betas_fastgrad[i, :], x, y)
    fig, ax = plt.subplots()
    ax.plot(range(1, niter + 1), error_grad, label='gradient descent')
    ax.plot(range(1, niter + 1), error_fastgrad, c='red', label='fast gradient')
    plt.xlabel('Iteration')
    plt.ylabel('Misclassification error')
    if title:
        plt.title(title)
    ax.legend(loc='upper right')
    if not save_file:
        plt.show()
    else:
        plt.savefig(save_file)

lr_cv = sklearn.linear_model.LogisticRegressionCV(penalty='l2', fit_intercept=False, tol=10e-8, max_iter=1000)
lr_cv.fit(x_train, y_train)
optimal_lambda = lr_cv.C_[0]
print('Optimal lambda=', optimal_lambda)

betas_grad = graddescent(beta_init, optimal_lambda, eta_init, maxiter)
betas_fastgrad = fastgradalgo(beta_init, theta_init, optimal_lambda, eta_init, maxiter)









HTTPError: HTTP Error 404: Not Found